## Building an Intelligent Restaurant Recommender with MCP and Mistral Large 2

In this notebook, we'll build a personalized restaurant recommendation system using Mistral Large 2 with the Model Context Protocol (MCP). Our system will consider user preferences stored in memory to provide tailored restaurant recommendations.

MCP (Model Context Protocol) is an open protocol that standardizes how LLMs interact with external data sources and tools. Think of MCP as a "universal connector" that lets AI models plug into different services while maintaining consistent interfaces.

We'll use three MCP servers:

1. **[Google Maps MCP Server](https://github.com/modelcontextprotocol/servers/tree/main/src/google-maps)**: For finding restaurants, getting details, and understanding locations
2. **[Time MCP Server](https://github.com/modelcontextprotocol/servers/tree/main/src/time)**: To check local time and determine restaurant availability
3. **[Memory MCP Server](https://github.com/modelcontextprotocol/servers/tree/main/src/memory)**: To store and retrieve user preferences

### Getting a Google Maps API Key

To use the Google Maps MCP server, you'll need your own API key:

1. Go to the [Google Cloud Console](https://developers.google.com/maps/documentation/javascript/get-api-key#create-api-keys)
2. Create a new project or select an existing one
3. Follow the steps to create a new API key


Our system works as follows:

**User Request**: User asks for restaurant recommendations with location data

**Location Analysis**: Maps MCP server converts coordinates to a readable address

**Time Check**: Time MCP server determines local time to filter for open restaurants

**Preference Retrieval**: Memory MCP server provides user's restaurant preferences

**Restaurant Search**: Maps MCP searches for restaurants matching preferences

**Detail Enrichment**: Get specific details about promising restaurant options

**Recommendation**: Format personalized recommendations based on user preferences


## ⚠️ Environment Note
**This notebook is optimized for Jupyter Lab environment.**

While this code can run in other environments (like SageMaker Studio), you may encounter async communication issues. For the best experience:
- Run this in Jupyter Lab
- Ensure all dependencies are properly installed
- Make sure Node.js and npm are configured in your environment

If you encounter any issues in other environments, try using the alternative time server configuration:
```python
"time": StdioServerParameters(
    command="python",
    args=["-m", "mcp_server_time"]
)



## Setting Up MCP Servers

To set up our MCP servers, we need to install several components:

### Required Packages
```python
!pip install -q mcp boto3 nest-asyncio uv mcp-server-time && npm install -g @modelcontextprotocol/server-google-maps @modelcontextprotocol/server-memory
```
This command installs:

mcp: The MCP client library

boto3: AWS SDK for Python

nest-asyncio: Enables async operations in Jupyter
                     
uv: Required for the time server
                     
mcp-server-time: The Time MCP server
                     
@modelcontextprotocol/server-google-maps: The Google Maps MCP server
                     
@modelcontextprotocol/server-memory: The Memory MCP server


In [ ]:
!pip install -q mcp boto3 nest-asyncio uv mcp-server-time && npm install -g @modelcontextprotocol/server-google-maps @modelcontextprotocol/server-memory

In [ ]:
import boto3
import json
from typing import List, Dict, Any

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from contextlib import AsyncExitStack
import asyncio
import nest_asyncio

In [ ]:
nest_asyncio.apply()

In [ ]:
bedrock = boto3.client('bedrock-runtime')
MISTRAL_MODEL = 'mistral.mistral-large-2407-v1:0'

## The RestaurantRecommender class handles:

**Server Connections**: Establishes connections to all three MCP servers

**Tool Routing**: Routes tool calls to the appropriate MCP server

**Query Processing**: Manages the conversation with Mistral Large 2, handling tool requests

**Memory Management**: Sets up and retrieves user preferences


Our notebook initializes these servers using the MCP client library, with specific configurations for each:

In [10]:
class RestaurantRecommender:
    def __init__(self):
        self.exit_stack = AsyncExitStack()
        self.sessions = {}
        
    async def connect_to_mcp_servers(self):
        """Initialize connections to MCP servers with improved error handling"""
        
        servers_config = {
            "maps": StdioServerParameters(
                command="npx",
                args=["@modelcontextprotocol/server-google-maps"],
                env={"GOOGLE_MAPS_API_KEY": "Your_API_Key"}
            ),
            "time": StdioServerParameters(
                command="uvx",  # Using uvx as recommended
                args=["mcp-server-time"]
            ),
            "memory": StdioServerParameters(
                command="npx",
                args=["@modelcontextprotocol/server-memory"]
            )
        }
        
        for name, params in servers_config.items():
            try:
                print(f"Connecting to {name} server...")
                stdio_transport = await self.exit_stack.enter_async_context(
                    stdio_client(params)
                )
                stdio, write = stdio_transport
                session = await self.exit_stack.enter_async_context(
                    ClientSession(stdio, write)
                )
                self.sessions[name] = session
                await session.initialize()
                
                # Test the connection by getting available tools
                tools = await session.list_tools()
                print(f"Successfully connected to {name} server. Available tools:")
                # Modified this part to handle tuple format
                for tool in tools.tools:  # Access the tools attribute
                    print(f"  - {tool.name}: {tool.description}")
                
                # For time server, let's test it specifically
                if name == "time":
                    try:
                        current_time = await session.call_tool("get_current_time", {"timezone": "UTC"})
                        print(f"Time server test: Current UTC time is {current_time.content}")
                    except Exception as e:
                        print(f"Time server test failed: {str(e)}")
                
            except Exception as e:
                print(f"Error connecting to {name} server: {str(e)}")
                print("Stack trace:")
                import traceback
                traceback.print_exc()
                raise

    async def setup_memory(self):
        """Initialize memory with default user preferences"""
        try:
            await self.sessions["memory"].call_tool(
                "create_entities",
                {
                    "entities": [
                        {
                            "name": "default_user",
                            "entityType": "person",
                            "observations": [
                                "Prefers Japanese and sushi restaurants",
                                "Typically spends $$ on meals",
                                "Prefers restaurants within 2 miles",
                                "Enjoys trying new ramen places",
                                "Vegetarian options are important"
                            ]
                        }
                    ]
                }
            )
            print("Memory initialized with user preferences")
        except Exception as e:
            print(f"Error setting up memory: {str(e)}")
            raise

    async def get_available_tools(self) -> List[Dict]:
        """Get list of available tools from all MCP servers"""
        tools = []
        for server_name, session in self.sessions.items():
            try:
                response = await session.list_tools()
                tools.extend([{
                    "name": t.name,
                    "description": t.description,
                    "input_schema": t.inputSchema
                } for t in response.tools])  # Access response.tools instead of response
            except Exception as e:
                print(f"Error getting tools from {server_name}: {str(e)}")
        return tools

    async def execute_tool(self, tool_name: str, arguments: Dict) -> str:
        tool_prefix_map = {
            "maps_": "maps",
            "get_current_time": "time",
            "convert_time": "time", 
            "search_nodes": "memory",
            "open_nodes": "memory",
            "create_entities": "memory",
            "create_relations": "memory",
            "add_observations": "memory",
            "delete_entities": "memory",
            "delete_observations": "memory",
            "delete_relations": "memory",
            "read_graph": "memory"
        }
        
        # First try to route based on tool prefix
        server_name = None
        for prefix, server in tool_prefix_map.items():
            if tool_name == prefix or tool_name.startswith(prefix):
                server_name = server
                break
        
        if server_name and server_name in self.sessions:
            try:
                print(f"Executing {tool_name} on {server_name} server")
                result = await self.sessions[server_name].call_tool(tool_name, arguments)
                return result.content
            except Exception as e:
                print(f"Error executing {tool_name} on {server_name} server: {str(e)}")
                raise ValueError(f"Error executing {tool_name}: {str(e)}")
        
        
        if not server_name:
            for name, session in self.sessions.items():
                try:
                    print(f"Trying {tool_name} on {name} server")
                    result = await session.call_tool(tool_name, arguments)
                    return result.content
                except Exception as e:
                    print(f"Failed on {name} server: {str(e)}")
                    continue
        
        raise ValueError(f"Tool {tool_name} not found in any MCP server")

    async def process_query(self, query: str, lat: float, lng: float):
        """Process a query using Mistral with tool access"""
        try:
            # Initialize conversation with just the user query
            conversation = [{
                "role": "user",
                "content": [{"text": f"Find restaurants near latitude {lat}, longitude {lng}. {query}"}]
            }]
            
            # Set up tools
            raw_tools = await self.get_available_tools()
            formatted_tools = []
            for tool in raw_tools:
                formatted_tools.append({
                    "toolSpec": {
                        "name": tool["name"],
                        "description": tool["description"],
                        "inputSchema": {
                            "json": tool["input_schema"]
                        }
                    }
                })
            
            # Format system correctly
            system = [{"text": SYSTEM_PROMPT}]
            
            while True:
                print(f"Sending request to Bedrock with {len(conversation)} messages in conversation")
                
                # Make API call
                response = bedrock.converse(
                    modelId=MISTRAL_MODEL,
                    messages=conversation,
                    system=system,
                    toolConfig={
                        "tools": formatted_tools,
                        "toolChoice": {"auto": {}}
                    }
                )
                
                # Get response data
                output = response.get("output", {})
                message = output.get("message", {})
                content = message.get("content", [])
                stop_reason = response.get("stopReason", "none")
                
                print(f"Got response with stop reason: {stop_reason}")
                
                # Check if the model wants to use tools
                if stop_reason == "tool_use":
                    print("Model is requesting tool use")
                    
                    # Store the assistant's message with tool requests in our conversation
                    conversation.append({
                        "role": "assistant",
                        "content": content
                    })
                    
                    # Extract all tool use requests
                    tool_use_requests = []
                    for idx, item in enumerate(content):
                        if "toolUse" in item:
                            tool_use_requests.append(item["toolUse"])
                    
                    # Log the requests we've identified
                    print(f"Found {len(tool_use_requests)} tool requests")
                    for req in tool_use_requests:
                        print(f"  - {req.get('name')} (ID: {req.get('toolUseId')})")
                    
                    # Collect all tool results in the same order as they were requested
                    tool_results = []
                    
                    for req in tool_use_requests:
                        tool_name = req.get("name")
                        tool_input = req.get("input", {})
                        tool_id = req.get("toolUseId")
                        
                        print(f"Executing tool: {tool_name} with ID {tool_id}")
                        
                        try:
                            result = await self.execute_tool(tool_name, tool_input)
                            
                            # Convert complex result objects to string
                            if isinstance(result, list) and len(result) > 0:
                                if hasattr(result[0], 'text'):
                                    result = result[0].text
                            
                            print(f"Tool result: {str(result)[:100]}...")
                            
                            # Add to tool results collection
                            tool_results.append({
                                "toolResult": {
                                    "toolUseId": tool_id,
                                    "content": [{"text": str(result)}]
                                }
                            })
                            
                        except Exception as e:
                            print(f"Error executing {tool_name}: {e}")
                            tool_results.append({
                                "toolResult": {
                                    "toolUseId": tool_id,
                                    "content": [{"text": f"Error: {str(e)}"}],
                                    "status": "error"
                                }
                            })
                    
                    # Add all tool results in a single message
                    conversation.append({
                        "role": "user",
                        "content": tool_results  # List of all tool results
                    })
                    
                    print(f"Added {len(tool_results)} tool results to conversation")
                    
                    # Continue loop to make another API call with updated conversation
                    
                else:
                    # Model has provided a final answer
                    print("Model has provided a final answer")
                    
                    # Extract text from content
                    text_parts = []
                    for item in content:
                        if isinstance(item, dict) and "text" in item:
                            text_parts.append(item["text"])
                    
                    result = "\n".join(text_parts)
                    return result if result else "No text content found in response"
                    
        except Exception as e:
            print(f"Error in process_query: {str(e)}")
            import traceback
            traceback.print_exc()
            raise

## System Prompt Overview
The system prompt for our restaurant recommender instructs Mistral Large 2 to act as a personalized recommendation assistant with the following key elements:

**Available Tools**

**Google Maps Tools**: For location services and restaurant data

**maps_search_places**: Find restaurants by query and location

**maps_place_details**: Get detailed restaurant information

**maps_reverse_geocode**: Convert coordinates to addresses

**Time Tools**: For restaurant availability

**get_current_time**: Check current time in specific timezones

**convert_time**: Convert between timezones

**Memory Tools**: For personalization

**open_nodes**: Direct access to user preferences

**search_nodes**: Search for preference data

**Recommendation Process**

- Determine the user's location using reverse geocoding
- Check local time to filter for open restaurants
- Access user's dining preferences from memory
- Search for restaurants matching preferences
- Get detailed information about promising options
  
Memory Access Strategy

**Primary**: Use open_nodes with ["default_user"] for direct preference access

**Fallback**: Use search_nodes if direct access fails

**Relations**: Check the user's past dining experiences through entity relationships


In [ ]:
SYSTEM_PROMPT = """
    You are a restaurant recommendation assistant with access to the following MCP tools:

    Google Maps Tools:
    - maps_search_places: Search for restaurants by query, location, and radius
    - maps_place_details: Get detailed information about a specific place
    - maps_reverse_geocode: Convert coordinates to address information
    
    Time Tools:
    - get_current_time: Get current time in a specific timezone
    - convert_time: Convert time between timezones
    
    Memory Tools:
    - search_nodes: Access user preferences and dining history
    
    Process for making recommendations:
    1. Use maps_reverse_geocode to understand the area
    2. Use get_current_time to check local time and determine which restaurants are likely to be open
    3. Use search_nodes to get user preferences
    4. Use maps_search_places to find restaurants
    5. Use maps_place_details to get detailed information about promising options
    
    When recommending:
    - Always begin by using open_nodes with names ["default_user"] to directly access user preferences
    - If that fails, fall back to search_nodes with queries like "restaurant preferences" or "food preferences"
    - Consider user's previous dining experiences by checking relations where "default_user" is the source
    
    Format your response conversationally, explaining why each recommendation would suit the user.
    If you need any information, use the appropriate tool to get it."""

## Working with User Preferences

User preferences are stored in the Memory MCP server as entities with observations:

In [ ]:
async def setup_test_memory(recommender):
    """Set up test user preferences in Memory MCP"""
    
    # Create user entity with preferences
    await recommender.sessions["memory"].call_tool(
        "create_entities",
        {
            "entities": [
                {
                    "name": "default_user",
                    "entityType": "person",
                    "observations": [
                        "Prefers Japanese and sushi restaurants",
                        "Typically spends $$ on meals",
                        "Prefers restaurants within 2 miles",
                        "Enjoys trying new ramen places",
                        "Vegetarian options are important",
                        "restaurant preferences"
                    ]
                },
                {
                    "name": "Sushi_Delight",
                    "entityType": "restaurant",
                    "observations": [
                        "Japanese restaurant",
                        "Known for fresh sushi",
                        "Price level: $$",
                        "Last visited: 2024-02-15",
                        "restaurants"
                    ]
                },
                {
                    "name": "Ramen_House",
                    "entityType": "restaurant",
                    "observations": [
                        "Japanese ramen restaurant",
                        "Price level: $$",
                        "Known for vegetarian options",
                        "Last visited: 2024-03-01"
                    ]
                }
            ]
        }
    )
    
    # Create relations (previous experiences)
    await recommender.sessions["memory"].call_tool(
        "create_relations",
        {
            "relations": [
                {
                    "from": "default_user",
                    "to": "Sushi_Delight",
                    "relationType": "enjoyed_dining_at"
                },
                {
                    "from": "default_user",
                    "to": "Ramen_House",
                    "relationType": "frequently_visits"
                }
            ]
        }
    )

In [ ]:
async def main():
    try:
        print("1. Initializing recommender...")
        recommender = RestaurantRecommender()
        
        print("2. Connecting to MCP servers...")
        await recommender.connect_to_mcp_servers()
        
        print("3. Setting up test memory...")
        await setup_test_memory(recommender)
        
        print("4. Getting available tools...")
        tools = await recommender.get_available_tools()
        print(f"Available tools: {tools}")
        
        # Example coordinates - feel free to set your own!
        lat, lng = 40.7484, -73.985428
        
        # Example query
        query = "I'm looking for dinner recommendations. Something similar to places I've enjoyed before."
        
        print("5. Processing query...")
        response = await recommender.process_query(query, lat, lng)
        print("6. Got response!")
        print(response)
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        raise e

In [ ]:
await main()